## Import the data

Use Pandas to import the data and prepare it for the models

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import plotly.express as px


df = pd.read_csv('Dataset/dataset_phishing.csv')

#print null
print(df.isnull().sum())

#remove all null
df = df.dropna()

#Trying absolute value on domain_age
df.domain_age = df.domain_age.abs()
print(f"Minimum domain age: {np.min(df.domain_age)}")

#describe statistical data. To stdout
with pd.option_context('display.max_columns', 40):
    print(df.describe(include='all'))

#describe statistical data to text file: out.txt
with open('out.txt', 'w') as f:
    with pd.option_context('display.max_columns', 40):
        print(df.describe(include='all'),file=f)
        
#shape of data
print('Number of rows are',df.shape[0], 'and number of columns are ',df.shape[1])

#look at data types of columns
print(df.info())



#encode last column 
dummy_data = pd.get_dummies(df, columns = ['status'])

#describe statistical data. To stdout
print("Here is the dummy encoded data")
with pd.option_context('display.max_columns', 40):
    print(dummy_data.describe(include='all'))


#Get X and y 
X = dummy_data.iloc[:,1:-1].values
y = dummy_data.iloc[:,-1].values

#try printing last column to make sure it's binary
print("Here are the labels")
print(y)
print("Here is X")
print(X)
print(X.shape, y.shape)

#Shows that the data set is balanced. 50% 50% 
print(f"Here are the number of legitamate URLs {np.sum(y==0)}. Or {np.sum(y==0)/y.shape[0]}%")
print(f"Here are the number of phishing URLs {np.sum(y==1)}. Or {np.sum(y==1)/y.shape[0]}%")

#Train, test, val split. 60%, 20%, 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

#Doing test_train split for now, later cross validation
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
print(f"X_train shape = {X_train.shape}, X_val shape = {X_val.shape}, X_test shape = {X_test.shape}")
print(f"y_train shape = {y_train.shape}, y_val shape = {y_val.shape}, y_test shape = {y_test.shape}")

## Random Forest Model

In [ ]:
#Need to do feature scaling
scaler = StandardScaler()
X_trains = scaler.fit_transform(X_train)
#just transform because we already did a fit above ^
X_tests = scaler.transform(X_test)
X_vals = scaler.transform(X_val)

#Random Forest
#default = 100
estimators = [10, 100, 500]
#default = None
max_leaf = [10, 50, None]

random_state = [0,20,40]
bestModel = 0
bestScore = 0
bestParams = [0,0,0]

#parellel arrays, so I can have the best, and also some worse models
scores = []
models = []


#will probably want to do some learning curves here 
for e in estimators:
    for m in max_leaf:
        for r in random_state:
            model = ensemble.RandomForestClassifier(n_estimators=e, max_leaf_nodes=m, random_state=r)
            model.fit(X_trains,y_train.flatten())
            print(f"Training Score: {model.score(X_trains,y_train)}")
            # it makes predictions using X_test under the hood and uses those predictions to calculate accuracy score
            score = model.score(X_vals,y_val)
            models.append(model)
            scores.append(score)
            print(f"Validation Score: n_estimators = {e}, max_leaf = {m} , random = {r}, Score = {score}")
            if score > bestScore:
                bestModel = model
                bestScore = score
                bestParams = [e,m,r]
                
            


scores = np.array(scores)
models = np.array(models, dtype=object)

#sorts in ascending order
bestIndices = np.argsort(scores)
ascendingModels = []
for i in bestIndices:
    ascendingModels.append(models[i])
    
# print(ascendingModels[-1])
bestModel2 = ascendingModels[-1]
# print(ascendingModels[0])
worstModel = ascendingModels[0]

print(f"Worst Model: n_estimators = {worstModel.n_estimators}, max_leaf = {10} , random = {worstModel.random_state}, Score = {worstModel.score(X_vals,y_val)}")
#check test score of best model
print(f"Worst Model Test Score: {worstModel.score(X_tests, y_test)}")

print(f"Best Model: n_estimators = {bestParams[0]}, max_leaf = {bestParams[1]} , random = {bestParams[2]}, Score = {bestScore}")
#check test score of best model
print(f"Best Model Test Score: {bestModel.score(X_tests, y_test)}")

print(bestModel2)
print(f"Best Model2: n_estimators = {bestModel2.n_estimators}, max_leaf = {None} , random = {bestModel2.random_state}, Score = {bestModel2.score(X_vals,y_val)}")
#check test score of best model
print(f"Best Model Test Score: {bestModel2.score(X_tests, y_test)}")

    



## Learning Curves for Random Forest

In [ ]:
# #Learning curve for best model (and maybe try another model or 2?)
from sklearn.metrics import mean_squared_error

# reccomended - a function to compute data and draw the learning curves
def plot_learning_curves(model, Xtrain, ytrain, Xval, yval, ax):
    
    mTrain = Xtrain.shape[0]
    
    #will be used to graph later
    Jtrain = []
    Jval = []
    
    trainingExamples = mTrain
    
    #loop up to mTrain, taking small subsets of the train data set from 1-mTrain
    for m in range(1,trainingExamples, 100):
        #create subset of training data
        XTrainTemp = Xtrain[0:m,:]
        yTrainTemp = ytrain[0:m]

        model.fit(Xtrain, ytrain.flatten())
        yTrainPred = model.predict(XTrainTemp)
        #test validation on the whole data set
        yValPred = model.predict(Xval)
#         print(yTrainTemp.shape)
#         print(yTrainPred.shape)
        
        #squared = False -> RMSE rather than MSE
        Jtrain.append(mean_squared_error(yTrainTemp,yTrainPred, squared=False ))
        #also need to test validation on the whole set
        Jval.append(mean_squared_error(yval,yValPred, squared=False ))
    #plot
    ax.set_xlabel('Training Set Size')
    ax.set_ylabel('RMSE')
    i = [int(x) for x in range(1,trainingExamples, 100)]
    ax.plot(i,Jtrain,"r-",label="train")
    ax.plot(i,Jval,"b-",label="val")
    ax.legend(loc = 'best')
    
fig = plt.figure(figsize=(14, 5))
fig.subplots_adjust(wspace=0.3)
ax1 = fig.add_subplot(131)
ax1.set_title("Worst Model")
ax2 = fig.add_subplot(132)
ax2.set_title("Best Model")
ax3 = fig.add_subplot(133)
ax3.set_title("Best Model 2")
ax2.set_ylim(0,0.05);
ax3.set_ylim(0,0.02);
# ax3.set_ylim(0,200);
    
#Learning Curvues for Random Forest 

#worst model, a litle underfitting
plot_learning_curves(worstModel, X_trains, y_train, X_vals, y_val, ax1)
#next 2 are both 100%
plot_learning_curves(bestModel, X_trains, y_train, X_vals, y_val, ax2)
plot_learning_curves(bestModel2, X_trains, y_train, X_vals, y_val, ax3)


    



## Precision/Recall/F1 Score 

In [ ]:
yWorst = worstModel.predict(X_tests)
yBest = bestModel.predict(X_tests)
yBest2 = bestModel2.predict(X_tests)


print(f"Worst Model: Precision = {precision_score(y_test,yWorst)},\
      Recall = {recall_score(y_test,yWorst)}, f1score = {f1_score(y_test,yWorst)}")
print(f"Best Model: Precision = {precision_score(y_test,yBest)},\
      Recall = {recall_score(y_test,yBest)}, f1score = {f1_score(y_test,yBest)}")
print(f"Best Model2: Precision = {precision_score(y_test,yBest2)},\
      Recall = {recall_score(y_test,yBest2)}, f1score = {f1_score(y_test,yBest2)}")

## Feature Importance for Random Forest

In [ ]:
#probabilities that determine each feature's importance
featureImportance = bestModel2.feature_importances_

sortedIndices = np.argsort(featureImportance)
descendingIndices = np.flip(sortedIndices,0)
#probabilities in descending  order
featureImportanceDescending = featureImportance[descendingIndices]
#a list that goes from best -> worst features
bestFeatures = df.columns[descendingIndices]
    
print("Here are the 10 best features")
print(bestFeatures[:10])
    
bestFeatures = bestFeatures[:10]
featureImportanceDescending = featureImportanceDescending[:10]

#BAR GRAPH OF BEST FEATURES
plt.figure(figsize=(10,5))
chart = sns.barplot(
    x=bestFeatures, y=featureImportanceDescending
)
plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='medium'  
)
chart.set(title="Best 10 Features for Best Model 2")
plt.show()

## Data Visualization With Best Features

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# plt.figure(figsize=(20,20))
#Just take best 10
bestFeatures = bestFeatures[:5]
# print(len(bestFeatures))
sns.pairplot(df, vars = bestFeatures)
plt.show()

#Best 2 features on scatter plot

sns.scatterplot(x=df.page_rank, y=df.dns_record, hue = df.status)
plt.show()

#Best 3 features on 3d plot
sns.set(style = "darkgrid")

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = df.page_rank
y = df.dns_record
z = df.google_index

ax.set_xlabel("page_rank")
ax.set_ylabel("dns_record")
ax.set_zlabel("google_index")
ax.scatter(x, y, z)
plt.show()


#histogram for phishing vs. legititmate
sns.countplot(df['status'])
plt.show()

sns.countplot(x=df.google_index, hue=df.status)
plt.show()

#Page rank
sns.displot(df.page_rank, color = 'red', label = 'page_rank', kde = True)
plt.legend
plt.show()


#Page rank vs. Phishing
plt.figure(figsize=(10,10))
sns.displot(df,x="page_rank",hue="status",kind = "kde")
plt.title('Phishing versus Page Rank')
plt.show()

#DNS Record vs Phishing
plt.figure(figsize=(10,10))
sns.displot(df,x="dns_record",hue="status",kind = "kde")
plt.title('Phishing versus DNS Record')
plt.show()

plt.figure(figsize=(10,10))
sns.displot(df,x="google_index",hue="status",kind = "kde")
plt.title('Phishing versus Google Index')
plt.show()

# plt.figure(figsize=(10,10))
# sns.displot(df,x="ratio_intRedirection",hue="status",kind = "kde")
# plt.title('Phishing versus Ratio Int Redirection')
# plt.show()

plt.figure(figsize=(10,10))
sns.displot(df,x="domain_age",hue="status",kind = "kde")
plt.title('Phishing versus Domain Age')
plt.show()

plt.figure(figsize=(10,10))
sns.displot(df,x="popup_window",hue="status",kind = "kde")
plt.title('Phishing versus popup_window')
plt.show()

plt.figure(figsize=(10,10))
sns.displot(df,x="avg_word_path",hue="status",kind = "kde")
plt.title('Phishing versus Avg Word Path')
plt.show()

plt.figure(figsize=(10,10))
sns.displot(df,x="nb_space",hue="status",kind = "kde")
plt.title('Phishing versus # Space')
plt.show()

# plt.figure(figsize=(10,10))
# sns.displot(df,x="url",hue="status",kind = "kde")
# plt.title('Phishing versus url')
# plt.show()

## Clustering

In [ ]:
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.io as pio
# Import the necessaries libraries
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline
pyo.init_notebook_mode()
indexes = [df.columns.get_loc('page_rank')-1,df.columns.get_loc('dns_record')-1,df.columns.get_loc('google_index')-1]
print(indexes)

#Make array of two best features, already scaled
TwoBestTrain = X_trains[:,[indexes[0],indexes[1]]]
TwoBestTest = X_tests[:,[indexes[0],indexes[1]]]

ThreeBestTrain = X_trains[:,indexes]
ThreeBestTest = X_tests[:,indexes]

km1 = KMeans(n_clusters=2).fit(TwoBestTrain)
km2  = KMeans(n_clusters=2).fit(ThreeBestTrain)


# Check proportion of positive and negative samples
yPred = km1.predict(TwoBestTest)
print("Two Best")
print(f"Number of predictions for cluster center = 0: {np.sum(yPred == 0)}")
print(f"Number of predictions for cluster center = 1: {np.sum(yPred == 1)}")

yPred = km2.predict(ThreeBestTest)
print("Three Best")
print(f"Number of predictions for cluster center = 0: {np.sum(yPred == 0)}")
print(f"Number of predictions for cluster center = 1: {np.sum(yPred == 1)}")

## Plot ThreeBest and cluster centers

fig = px.scatter_3d(x=ThreeBestTest[:,0], y=ThreeBestTest[:,1], z=ThreeBestTest[:,2])
fig.show()



## SVM

In [ ]:
svmS = StandardScaler()
X_trains = svmS.fit_transform(X_train)
X_vals = svmS.transform(X_val)

#TODO is not outputting best params, just last params

#literally no change in val when C > 2, 2 is the max
cVals = [0.01, 0.03, 0.1, 0.3, 1, 2]
#Try with higher gamma
gammaVals = [1, 5, 20, 50, 100, 200]
score = 0
bestParams = []
#Linear kernel
print("Values for linear kernel")
for i in gammaVals:
    gamma = 1/(i**2)
    for j in cVals:
        model = svm.SVC(C=j, gamma=gamma)
        model.fit(X_trains, y_train.flatten())
        if model.score(X_vals, y_val.flatten()) > score:
            bestParams = ["linear", j, i]
        print(f"Score for linear kernel with C = {j} and sigma = {gamma}: Train data - {model.score(X_trains, y_train.flatten())} Val data - {model.score(X_vals, y_val.flatten())}")
        
#rbf kernel
print("\nValues for rbf kernel")
for i in gammaVals:
    gamma = 1/(i**2)
    for j in cVals:
        model = svm.SVC(kernel='rbf', C=j, gamma=gamma)
        model.fit(X_trains, y_train.flatten())
        if model.score(X_vals, y_val.flatten()) > score:
            bestParams = ["rbf", j, i]
        print(f"Score for rbf kernel with C = {j} and sigma = {gamma}: Train data - {model.score(X_trains, y_train.flatten())} Val data - {model.score(X_vals, y_val.flatten())}")
        
print(f"\nBest params: kernel = {bestParams[0]}, C = {bestParams[1]}, gamma = {bestParams[2]}")

## Cross Validation on Random Forest Model

In [ ]:
for e in estimators[:-2]:
    for m in max_leaf[:-2]:
        for r in random_state[:-2]:
            model = ensemble.RandomForestClassifier(n_estimators=e, max_leaf_nodes=m, random_state=r)
            model.fit(X_train,y_train.flatten())
            # it makes predictions using X_test under the hood and uses those predictions to calculate accuracy score
            score = cross_val_score(model, X, y.flatten(), cv=5)
            print(f"model - estimators = {e}; max_leaf = {m}; random_state = {r}:{score}")

## Logistic Regression Model

In [ ]:
logRS = StandardScaler()
X_trains = logRS.fit_transform(X_train)
X_vals = logRS.transform(X_val)

cVals = [0.01, 0.03, 0.1, 0.3, 1, 2, 5, 10]

score = -1
bestParams = []
#Logistic regression with l2
print("Logistic regression using l2 penalty:")
for i in cVals:
    model = linear_model.LogisticRegression(C=i, solver='saga')
    model.fit(X_trains, y_train)
    print(f"Logistic regression using l2 and C={i}: Train - {model.score(X_trains, y_train)}; Val - {model.score(X_vals, y_val)}")
    if model.score(X_vals, y_val) > score or score == -1:
        score = model.score(X_vals, y_val)
        bestParams = ['l2', i]
    

#Logistic regression with l1
print("\nLogistic regression using l1 penalty:")
for i in cVals:
    model = linear_model.LogisticRegression(C=i, penalty='l1', solver='saga')
    model.fit(X_trains, y_train)
    print(f"Logistic regression using l1 and C={i}: Train - {model.score(X_trains, y_train)}; Val - {model.score(X_vals, y_val)}")
    if model.score(X_vals, y_val) > score or score == -1:
        score = model.score(X_vals, y_val)
        bestParams = ['l1', i]
        
#Logistic regression with elastic net:
print("\nLogistic regression using elastic net:")
for i in cVals:
    model = linear_model.LogisticRegression(C=i, penalty='elasticnet', solver='saga', l1_ratio=0.5)
    model.fit(X_trains, y_train)
    print(f"Logistic regression using elastic net and C={i}: Train - {model.score(X_trains, y_train)}; Val - {model.score(X_vals, y_val)}")
    if model.score(X_vals, y_val) > score or score == -1:
        score = model.score(X_vals, y_val)
        bestParams = ['elasticnet', i]
        
print(f"\nThe best performing model uses the {bestParams[0]} penalty with C={bestParams[1]}")